In [1]:
from typing import Any, Optional
import numpy as np
import os
import pymongo
import pyarrow as pa
import ray
import pandas as pd
from openai import OpenAI
from pyarrow import csv
from pymongo import MongoClient, ASCENDING, DESCENDING, UpdateOne
from pymongo.operations import SearchIndexModel, IndexModel

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from ray.util.accelerators import NVIDIA_TESLA_A10G

2024-06-26 02:29:31,091	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.2 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ModuleNotFoundError: No module named 'openai'

In [2]:
from accelerate import  Accelerator
import torch
torch.cuda.empty_cache()
Accelerator().free_memory()

[]

In [ ]:
import argparse
import os
import subprocess

import torch
from PIL import Image

from vllm import LLM
from vllm.multimodal.image import ImageFeatureData, ImagePixelData

In [ ]:
# llm = LLM(
#     model="llava-hf/llava-1.5-7b-hf",
#     image_input_type="pixel_values",
#     image_token_id=32000,
#     image_input_shape="1,3,336,336",
#     image_feature_size=576,
#     disable_image_processor=False,
# )
llm = LLM(
    model="llava-hf/llava-v1.6-mistral-7b-hf",
    # image_input_type="pixel_values",
    # image_token_id=32000,
    # image_input_shape="1,3,336,336",
    # image_feature_size=576,
    # disable_image_processor=False,
    **{
        "trust_remote_code": True,
        "enable_lora": False,
        "max_num_seqs": 1,
        "max_model_len": 14224,
        "gpu_memory_utilization": 0.85,
        "image_input_type": "pixel_values",
        "image_token_id": 32000,
        "image_input_shape": "1,3,336,336",
        "image_feature_size": 1176,
        "enforce_eager": True,
    }
)

In [ ]:
disable_image_processor=False

prompt = "<image>" * 1176 + (
    "\nUSER: What is the content of this image?\nASSISTANT:")

if disable_image_processor:
    image = torch.load("images/stop_sign_pixel_values.pt")
else:
    image = Image.open("stop_sign.jpg")


In [ ]:
outputs = llm.generate({
    "prompt": prompt,
    "multi_modal_data": ImagePixelData(image),
})


In [ ]:
outputs[0].outputs

In [ ]:
# from mistral_inference.model import Transformer
# from mistral_inference.generate import generate

# from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
# from mistral_common.protocol.instruct.messages import UserMessage
# from mistral_common.protocol.instruct.request import ChatCompletionRequest


# tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
# model = Transformer.from_folder(mistral_models_path)

# completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

# tokens = tokenizer.encode_chat_completion(completion_request).tokens

# out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
# result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

# print(result)

In [3]:
class MistralvLLM:
    def __init__(self):
        self.llm = LLM(
            model="TheBloke/Mistral-7B-v0.1-AWQ",
            max_model_len=4096,
            skip_tokenizer_init=False,
            gpu_memory_utilization=0.40,
            quantization="awq",
            dtype="half"
        )


In [4]:
llm = MistralvLLM()

WARNING 06-26 02:14:48 config.py:1222] Casting torch.bfloat16 to torch.float16.
WARNING 06-26 02:14:48 config.py:217] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-26 02:14:48 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='TheBloke/Mistral-7B-v0.1-AWQ', speculative_config=None, tokenizer='TheBloke/Mistral-7B-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=TheBloke/Mistral-7B-v0.1-AWQ)


/home/ray/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-26 02:14:48 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-26 02:14:48 selector.py:51] Using XFormers backend.
INFO 06-26 02:14:49 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-26 02:14:49 selector.py:51] Using XFormers backend.
INFO 06-26 02:14:49 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 06-26 02:14:49 weight_utils.py:261] No model.safetensors.index.json found in remote.
INFO 06-26 02:14:50 model_runner.py:160] Loading model weights took 3.8814 GB
INFO 06-26 02:14:52 gpu_executor.py:83] # GPU blocks: 2140, # CPU blocks: 2048
INFO 06-26 02:14:55 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-26 02:14:55 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are runnin

In [5]:
sampling_params = SamplingParams(
    n=1,
    presence_penalty=0,
    frequency_penalty=0,
    repetition_penalty=1,
    length_penalty=1,
    top_p=1,
    top_k=-1,
    temperature=0,
    use_beam_search=False,
    ignore_eos=False,
    max_tokens=2048,
    seed=None,
    detokenize=False,
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
            "TheBloke/Mistral-7B-v0.1-AWQ",
            # "mistralai/Mistral-7B-Instruct-v0.1",
        )
input_text = "What is the capital of france?"
prompt_template=f'''{input_text}

'''

print("\n\n*** Generate:")

input_tokens = tokenizer(
    prompt_template,
    return_tensors='np'
).input_ids

# input_tokens = tokenizer.apply_chat_template(
#     conversation=[{"role": "user", "content": input_text}],
#     # add_generation_prompt=True,
#     tokenize=True,
#     return_tensors="np",
# ).squeeze()
# print(input_tokens)
input_tokens



*** Generate:


array([[    1,  1824,   349,   272,  5565,   302,  1104,   617, 28804,
           13,    13]])

In [11]:
# response = llm.llm.generate(prompt_token_ids=input_tokens.tolist(), sampling_params=sampling_params)
sampling_params = SamplingParams(
    n=1,
    presence_penalty=0,
    frequency_penalty=0,
    repetition_penalty=1,
    length_penalty=1,
    top_p=1,
    top_k=-1,
    temperature=0,
    use_beam_search=False,
    ignore_eos=False,
    max_tokens=2048,
    seed=None,
    detokenize=True,
)
response = llm.llm.generate(prompts=input_text, sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:25<00:00, 25.36s/it, est. speed input: 0.35 toks/s, output: 80.77 toks/s]


In [12]:
response

[RequestOutput(request_id=1, prompt='What is the capital of france?', prompt_token_ids=[1, 1824, 349, 272, 5565, 302, 1104, 617, 28804], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='', token_ids=[13, 13, 3916, 278, 349, 272, 5565, 302, 4843, 28723, 661, 349, 272, 7639, 2990, 297, 4843, 304, 272, 1080, 1852, 9504, 2990, 297, 272, 6392, 7085, 28723, 5465, 349, 835, 272, 1080, 11542, 2990, 297, 272, 1526, 28725, 395, 754, 28705, 28740, 28782, 3841, 25313, 13726, 272, 2990, 1012, 879, 28723, 13, 13, 1064, 1824, 349, 272, 5565, 302, 4843, 28804, 13, 13, 1014, 5565, 302, 4843, 349, 5465, 28723, 13, 13, 1064, 1824, 349, 272, 5565, 302, 4843, 28804, 13, 13, 1014, 5565, 302, 4843, 349, 5465, 28723, 13, 13, 1064, 1824, 349, 272, 5565, 302, 4843, 28804, 13, 13, 1014, 5565, 302, 4843, 349, 5465, 28723, 13, 13, 1064, 1824, 349, 272, 5565, 302, 4843, 28804, 13, 13, 1014, 5565, 302, 4843, 349, 5465, 28723, 13, 13, 1064, 1824, 349, 272, 5565, 302, 4843, 28804, 13, 13, 1014, 5565, 302

In [13]:
response[0].outputs[0].token_ids

[13,
 13,
 3916,
 278,
 349,
 272,
 5565,
 302,
 4843,
 28723,
 661,
 349,
 272,
 7639,
 2990,
 297,
 4843,
 304,
 272,
 1080,
 1852,
 9504,
 2990,
 297,
 272,
 6392,
 7085,
 28723,
 5465,
 349,
 835,
 272,
 1080,
 11542,
 2990,
 297,
 272,
 1526,
 28725,
 395,
 754,
 28705,
 28740,
 28782,
 3841,
 25313,
 13726,
 272,
 2990,
 1012,
 879,
 28723,
 13,
 13,
 1064,
 1824,
 349,
 272,
 5565,
 302,
 4843,
 28804,
 13,
 13,
 1014,
 5565,
 302,
 4843,
 349,
 5465,
 28723,
 13,
 13,
 1064,
 1824,
 349,
 272,
 5565,
 302,
 4843,
 28804,
 13,
 13,
 1014,
 5565,
 302,
 4843,
 349,
 5465,
 28723,
 13,
 13,
 1064,
 1824,
 349,
 272,
 5565,
 302,
 4843,
 28804,
 13,
 13,
 1014,
 5565,
 302,
 4843,
 349,
 5465,
 28723,
 13,
 13,
 1064,
 1824,
 349,
 272,
 5565,
 302,
 4843,
 28804,
 13,
 13,
 1014,
 5565,
 302,
 4843,
 349,
 5465,
 28723,
 13,
 13,
 1064,
 1824,
 349,
 272,
 5565,
 302,
 4843,
 28804,
 13,
 13,
 1014,
 5565,
 302,
 4843,
 349,
 5465,
 28723,
 13,
 13,
 1064,
 1824,
 349,
 272,
 5565

In [14]:
print(
    tokenizer.decode(response[0].outputs[0].token_ids, skip_special_tokens=False)
)



Paris is the capital of France. It is the largest city in France and the most populous city in the European Union. Paris is also the most visited city in the world, with over 15 million tourists visiting the city every year.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital of France?

The capital of France is Paris.

## What is the capital o

In [ ]:
tokenizer.decode?